In [1]:
!pip install pandas numpy scikit-learn nltk

Defaulting to user installation because normal site-packages is not writeable
  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached scikit_learn-1.6.1-cp312-cp312-win_amd64.whl.metadata (15 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached scipy-1.15.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached regex-2024.11.6-cp312-cp312-win_amd64.whl.metadata (41 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl (11.5 MB)
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.6 MB 2.4 MB/s eta 0:00:06
   -- ------------------


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\rushi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
import pickle
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Resume.csv to Resume.csv


In [ ]:
df = pd.read_csv("Resume.csv")

In [ ]:
print(df.head())

         ID                                         Resume_str  \
0  16852973           HR ADMINISTRATOR/MARKETING ASSOCIATE\...   
1  22323967           HR SPECIALIST, US HR OPERATIONS      ...   
2  33176873           HR DIRECTOR       Summary      Over 2...   
3  27018550           HR SPECIALIST       Summary    Dedica...   
4  17812897           HR MANAGER         Skill Highlights  ...   

                                         Resume_html Category  
0  <div class="fontsize fontface vmargins hmargin...       HR  
1  <div class="fontsize fontface vmargins hmargin...       HR  
2  <div class="fontsize fontface vmargins hmargin...       HR  
3  <div class="fontsize fontface vmargins hmargin...       HR  
4  <div class="fontsize fontface vmargins hmargin...       HR  


In [ ]:
print(df.columns.tolist())

['ID', 'Resume_str', 'Resume_html', 'Category']


In [ ]:
if 'Resume_str' not in df.columns or 'Category' not in df.columns:
    print("Columns present in the DataFrame:", df.columns.tolist())
    raise ValueError("The dataset must have 'Resume_str' and 'Category' columns.")

In [ ]:
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\W', ' ', text)  # Remove non-word characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    stop_words = set(stopwords.words('english'))
    words = [word for word in text.split() if word not in stop_words]
    return " ".join(words)


In [ ]:
df['Cleaned_Resume'] = df['Resume_str'].apply(clean_text)

In [ ]:
label_encoder = LabelEncoder()
df['Category_Encoded'] = label_encoder.fit_transform(df['Category'])

In [ ]:
with open("label_encoder.pkl", "wb") as le_file:
    pickle.dump(label_encoder, le_file)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df['Cleaned_Resume'], df['Category_Encoded'], test_size=0.2, random_state=42
)

In [ ]:
model = make_pipeline(TfidfVectorizer(), MultinomialNB())

In [ ]:
model.fit(X_train, y_train)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('multinomialnb', MultinomialNB())])

In [ ]:
with open("resume_classifier.pkl", "wb") as model_file:
    pickle.dump(model, model_file)

In [ ]:
print("✅ Model saved as resume_classifier.pkl")

✅ Model saved as resume_classifier.pkl


In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print("📊 Model Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

📊 Model Accuracy: 0.5251509054325956
                        precision    recall  f1-score   support

            ACCOUNTANT       0.79      0.90      0.84        29
              ADVOCATE       0.59      0.33      0.43        30
           AGRICULTURE       0.00      0.00      0.00         8
               APPAREL       0.75      0.15      0.25        20
                  ARTS       0.00      0.00      0.00        18
            AUTOMOBILE       0.00      0.00      0.00         6
              AVIATION       0.63      0.81      0.71        21
               BANKING       0.85      0.48      0.61        23
                   BPO       0.00      0.00      0.00         2
  BUSINESS-DEVELOPMENT       0.40      0.63      0.49        27
                  CHEF       0.81      0.71      0.76        24
          CONSTRUCTION       0.95      0.56      0.70        34
            CONSULTANT       0.50      0.05      0.09        20
              DESIGNER       0.78      0.74      0.76        19
  

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
def predict_resume(text):
    # Load trained model
    with open("resume_classifier.pkl", "rb") as model_file:
        loaded_model = pickle.load(model_file)
    # Load label encoder
    with open("label_encoder.pkl", "rb") as le_file:
        loaded_encoder = pickle.load(le_file)

        cleaned_text = clean_text(text)  # Clean input text
    prediction = loaded_model.predict([cleaned_text])  # Make prediction
    return loaded_encoder.inverse_transform(prediction)[0]  # Decode label

In [ ]:
new_resume = "I am a data scientist with experience in Python, machine learning, and deep learning."
print("📝 Predicted Category:", predict_resume(new_resume))

📝 Predicted Category: ENGINEERING


In [ ]:
new_resume = """
I am a software engineer with expertise in full-stack web development.
I have hands-on experience in JavaScript, React.js, Node.js, and Python.
My skills include designing scalable applications, building REST APIs,
and working with cloud technologies like AWS and Docker.
"""
print("📝 Predicted Category:", predict_resume(new_resume))

📝 Predicted Category: ENGINEERING


In [ ]:
new_resume = """
HR SPECIALIST       Summary    Dedicated, Driven, and Dynamic with over 20 years of customer service expertise. Motivated to maintain customer satisfaction and contribute to company success with an emphasis in personnel and process management. Solid team player with proven ability to establish rapport with clients.Â       Skills          Type 50 wpm and 10-Key by touch  Microsoft programs Word, Excel, Access, Outlook, PowerPoint  Personnel management  and Telephone inquiries specialist      Excellent customer service skills with call quality and customer satisfaction as primary focus.  Inbound and Outbound contact center experience  Excellent oral and written presentation and seasoned in conflict resolutionÂ             Experience      Hr Specialist     May 2007   to   Oct 2008      Company Name   ï¼   City  ,   State     Accenture Peopleline Email Management Team.  Received inbound emails from clients as initial point of contact to answer questions related to HR policies, practices or to resolve issues.  Helped the client by answering their inquiry, resolving their issue or routing their email to a caseworker for resolution.  Recorded and tracked the client issue via the case management tool from initiation to resolution.  Maintained records of customer interactions and transactions by documenting details of inquiries, complaints, and comments as well as actions taken.         Customer Service Representative     Mar 2007   to   Feb 2008      Company Name   ï¼   City  ,   State     Maintained records of customer interactions and transactions by documenting details of inquiries, complaints, and comments, as well as actions taken.  Developed and implemented filing and delivery methods for FMLA and Short Term Disability documentation to respective global clientele.  Facilitated communication for the Absence Management Team to clients and internal departmental teams.  Organized and maintained payroll and people services database management system.  Communicated to clients and internal/external departmental teams mitigated risks and resolutions regarding  policy, transactions and other queries         Call Centre Shift Supervisor     Aug 2003   to   Mar 2007      Company Name   ï¼   City  ,   State   Answered and Dispatched calls for Medical and Professional clients in the San Antonio and surrounding areas. Provided call monitoring to ensure staff was adhering to quality call control. Maintained On Call books for various medical staff to ensure that the correct staff was contacted during after hours and weekends.Â        Education and Training      Certificate of Completion
*Business Administration     1997     Texas A&M Extension Services   ï¼   City  ,   State            Skills    10-Key by touch, Type 50 wpm, filing, Access, Â Microsoft Excel, Microsoft Outlook, Microsoft PowerPoint, Microsoft Word
"""
print("📝 Predicted Category:", predict_resume(new_resume))

📝 Predicted Category: HR


In [ ]:
new_resume = """
   Summary     Software developer well-versed in the entire workflow for developing and implementing websites, including: consulting with the
   """
print("📝 Predicted Category:", predict_resume(new_resume)) # Removed the extra spaces before print

📝 Predicted Category: INFORMATION-TECHNOLOGY
